# Rekognition のだいたいの機能を使う
1. オブジェクトとシーンの検出
2. 画像の節度
3. 顔の分析
4. 有名人の認識
5. 顔の比較
6. イメージ内のテキスト
7. 顔のコレクション

---
やらないこと
1. Rekognition Video
2. Custom Labels

In [ ]:
import boto3,sagemaker,io
from PIL import Image

In [ ]:
client = boto3.client('rekognition')

In [ ]:
# データをダウンロード
!mkdir -p ./data/
!wget -O ./data/skateboard_resized.jpg https://dhei5unw3vrsx.cloudfront.net/images/skateboard_resized.jpg
!wget -O ./data/family_picnic.jpg https://dhei5unw3vrsx.cloudfront.net/images/family_picnic.jpg
!wget -O ./data/drive_resized.jpg https://dhei5unw3vrsx.cloudfront.net/images/drive_resized.jpg
!wget -O ./data/jeff_portrait.jpg https://dhei5unw3vrsx.cloudfront.net/images/jeff_portrait.jpg
!wget -O ./data/source3_resized.jpg https://dhei5unw3vrsx.cloudfront.net/images/source3_resized.jpg
!wget -O ./data/jeff_test.jpg https://assets.media-platform.com/bi/dist/images/2019/06/07/jeff-bezos-shares-his-best-advice-for-anyone-starting-a-business.jpg
!wget -O ./data/stive_test.jpg https://m.media-amazon.com/images/I/41b+TUx+W9L.jpg

# オブジェクトとシーンの検出 ( detect_labels )
1. バイト文字列 ( 5MB 以下)
2. S3

[詳細はこちら](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/rekognition.html#Rekognition.Client.detect_labels)

## 1. バイト文字列で detect_labels

In [ ]:
img_file = './data/skateboard_resized.jpg'

In [ ]:
with open(img_file,'rb') as f:
    img_bytes = f.read()
print(img_bytes[0:10])

In [ ]:
client.detect_labels(Image={
    'Bytes': img_bytes
})

## 2. S3 object でdetect_labels

In [ ]:
s3_location = sagemaker.Session().upload_data(img_file,bucket=sagemaker.Session().default_bucket(),key_prefix='asa_study')
print(s3_location)
bucket,key = sagemaker.s3.parse_s3_url(s3_location)
print(f'bucket: {bucket}')
print(f'key: {key}')

In [ ]:
client.detect_labels(Image={'S3Object':{
    'Bucket': bucket,
    'Name': key
}})

# 画像の節度 ( detect_moderation_labels )
1. バイト文字列 ( 5MB 以下)
2. S3

[詳細はこちら](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/rekognition.html#Rekognition.Client.detect_moderation_labels)

## 1. バイト文字列でdetect_moderation_labels

In [ ]:
# 5MB 以上の画像なので、バイト文字列で detect_moderation_labels する際はリサイズする
img_file = './data/family_picnic.jpg'
img = Image.open(img_file)
img_resize = img.resize((img.width//2,img.height//2))

img_resize = img.resize((img.width//2,img.height//2))
with io.BytesIO() as img:
    img_resize.save(img,format="jpeg")
    img_bytes = img.getvalue()#バイナリ取得

del img_resize,img

In [ ]:
print(img_bytes[0:10])

In [ ]:
%%time
client.detect_moderation_labels(Image={
    'Bytes': img_bytes
})

## 2. S3 で detect_moderation_labels

In [ ]:
%%time
client.detect_labels(Image={'S3Object':{
    'Bucket': 'console-sample-images-nrt',
    'Name': 'yoga_swimwear.jpg'
}})

# 顔の分析 ( detect_faces )
1. バイト文字列 ( 5MB 以下)
2. S3

[詳細はこちら](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/rekognition.html#Rekognition.Client.detect_faces)

In [ ]:
img_file = './data/drive_resized.jpg'
with open(img_file,'rb') as f:
    img_bytes = f.read()
print(img_bytes[0:10])

In [ ]:
response = client.detect_faces(
    Image={
        'Bytes': img_bytes,
    },
    Attributes=[
        'ALL',
    ]
)
print(response)

In [ ]:
%%time
response = client.detect_faces(Image={'S3Object':{
    'Bucket': 'console-sample-images-nrt',
    'Name': 'family.png'
}})
print(response)

# 有名人の認識 ( recognize_celebrities )
1. バイト文字列 ( 5MB 以下)
2. S3

[詳細はこちら](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/rekognition.html#Rekognition.Client.recognize_celebrities)

In [ ]:
img_file = './data/jeff_portrait.jpg'
with open(img_file,'rb') as f:
    img_bytes = f.read()
print(img_bytes[0:10])

In [ ]:
response = client.recognize_celebrities(
    Image={
        'Bytes': img_bytes
    }
)
print(response)

In [ ]:
%%time
response = client.get_celebrity_info(
    Id=response['CelebrityFaces'][0]['Id']
)
print(response)

In [ ]:
%%time
response = client.recognize_celebrities(
    Image={
        'S3Object': {
            'Bucket': 'console-sample-images-nrt',
            'Name': 'andy_portrait.jpg',
        }
    }
)
print(response)

# 顔の比較 ( compare_faces )
1. Sourceをbyte文字列
2. TargetをS3

In [ ]:
img_file = './data/source3_resized.jpg'
with open(img_file,'rb') as f:
    img_bytes = f.read()
print(img_bytes[0:10])

In [ ]:
response = client.compare_faces(
    SourceImage={
        'Bytes': img_bytes
    },
    TargetImage={
        'S3Object': {
            'Bucket': 'console-sample-images-nrt',
            'Name': 'target3.jpg',
        }
    },
    SimilarityThreshold=0
)
print(response)

# detect_text
* 以下バイト文字列省略

[詳細はこちら](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/rekognition.html#Rekognition.Client.detect_text)

In [ ]:
response = client.detect_text(
    Image={
        'S3Object': {
            'Bucket': 'console-sample-images-nrt',
            'Name': 'coffee_monday.jpg',
        }
    }
)
print(response)

# コレクション
1. コレクションを作成 (create_collection)
2. 顔を登録 (index_faces)
3. 顔イメージの検索
4. collection の削除

## 1. コレクションを作成

In [ ]:
# コレクション作成
COLLECTION_ID = 'MyFirstCollection'
response = client.create_collection(CollectionId=COLLECTION_ID)
print(response)

## 2. 顔を登録

In [ ]:
collection_id_name = {}

In [ ]:
# Jeff
response=client.index_faces(
    CollectionId=COLLECTION_ID,
    Image={
        'S3Object':{
            'Bucket':'console-sample-images-nrt',
            'Name':'jeff_portrait.jpg'
        }
    }
)
print(response)
collection_id_name[response['FaceRecords'][0]['Face']['FaceId']] = 'Jeff'

In [ ]:
# Andy
response=client.index_faces(
    CollectionId=COLLECTION_ID,
    Image={
        'S3Object':{
            'Bucket':'console-sample-images-nrt',
            'Name':'andy_portrait.jpg'
        }
    }
)
print(response)
collection_id_name[response['FaceRecords'][0]['Face']['FaceId']] = 'Andy'

## 顔イメージを検索

In [ ]:
img_file = './data/jeff_test.jpg'
with open(img_file,'rb') as f:
    img_bytes = f.read()
print(img_bytes[0:10])

In [ ]:
# Jeff で試す
response = client.search_faces_by_image(
    CollectionId=COLLECTION_ID,
    Image={
        'Bytes':img_bytes
    },
)
print(response)

In [ ]:
print(collection_id_name[response['FaceMatches'][0]['Face']['FaceId']])

In [ ]:
# Stive で試す,default th=0.8
img_file = './data/stive_test.jpg'
with open(img_file,'rb') as f:
    img_bytes = f.read()
response = client.search_faces_by_image(
    CollectionId=COLLECTION_ID,
    Image={
        'Bytes':img_bytes
    },
)
print(response)

## 4. collection の削除

In [ ]:
client.delete_collection(CollectionId = COLLECTION_ID)